# Former un tableau des structures non valides avec le nombre de documents associés

- Récupérer la liste des structures non valides
- La transformer en tableau (dataframe)
- Renommer les colonnes : `structId, label`
- Définir une fonction pour récupérer à partir du `structId` le nb de publication
- Ajouter la colonne nombre de documents
- Ajouter une colonne de liens vers la structure dans Aurehal
- Trier le tableau par ordre decroissant de nombre de documents
- Extraire le tableau

In [1]:
import requests, json
#uvsq 81173, U orléans 300297, UPCité 557826, angers 74911
structId = 557826

In [2]:
req = requests.get(f"https://api.archives-ouvertes.fr/ref/structure/?q=parentDocid_i:{structId}\
&fq=valid_s:INCOMING&rows=10000")
l_incoming = req.json()
l_incoming

{'response': {'numFound': 24,
  'start': 0,
  'numFoundExact': True,
  'docs': [{'docid': '1049198',
    'label_s': 'Service de Psychopathologie [Bobigny]'},
   {'docid': '1007985',
    'label_s': 'European Reference Network on respiratory diseases [ERN LUNG]'},
   {'docid': '1063040',
    'label_s': "Equipe Mobile d'Infectiologie [APHP Centre, Paris]"},
   {'docid': '1004389',
    'label_s': 'Reference Center for Constitutional Bone Diseases [Paris]'},
   {'docid': '1005164', 'label_s': 'Biophysique Cellulaire'},
   {'docid': '1004052',
    'label_s': 'Centre de ressources pour la remédiation cognitive et la réadaptation psychosociale [Hôpital Sainte-Anne - APHP] [C3RP]'},
   {'docid': '1071834', 'label_s': 'Equipe INSIGHT [Paris]'},
   {'docid': '1059686',
    'label_s': 'Biostatistique, Traitement et Modélisation des données biologiques'},
   {'docid': '1055760',
    'label_s': "Service de Psychiatrie et d'Addictologie de l'adulte et du sujet âgé [DMU Psychiatrie et Addictologie]"},

## Transformer la liste en tableau

In [3]:
import pandas as pd

In [5]:
struct_incoming = l_incoming["response"]["docs"]
df = pd.DataFrame(struct_incoming)
df[:4]

,docid,label_s
0,1049198,Service de Psychopathologie [Bobigny]
1,1007985,European Reference Network on respiratory dise...
2,1063040,"Equipe Mobile d'Infectiologie [APHP Centre, Pa..."
3,1004389,Reference Center for Constitutional Bone Disea...


In [6]:
# renomer les colonnes
df.columns = ["structId", "label"]
df.sample(3)

,structId,label
11,1079019,UFR des Sciences Fondamentales et Biomédicales...
19,1077254,Paris Centre University Hospitals
5,1004052,Centre de ressources pour la remédiation cogni...


## Definir une fonction pour récupérer à partir de `structId` le nb de publications

In [7]:
def req_hal(structId):
    req = requests.get(f"https://api.archives-ouvertes.fr/search/?q=structId_i:{structId}&rows=0")
    req = req.json()
    return req["response"]["numFound"]

In [8]:
# test
req_hal(49833)

2

## Ajouter la colonne nombre de documents à la DataFrame

In [9]:
df["nb_docs"] = df.apply(lambda row : req_hal(row["structId"]), axis = 1)
df.sample(5)

,structId,label,nb_docs
14,1008089,geoteca [geoteca],9
10,1077249,Pediatric and Perinatal Drug Evaluation and Ph...,1
18,1030919,Plastic Surgery Department [Paris],2
21,1004391,Department of Physical Medicine and Rehabilita...,2
0,1049198,Service de Psychopathologie [Bobigny],1


##  Ajouter une colonne de liens vers la structure dans Aurehal

In [9]:
df["aurehal_link"] = "https://aurehal.archives-ouvertes.fr/structure/read/id/" + df["structId"].apply(str)
df.sample(4)

,structId,label,nb_docs,aurehal_link
23,1004052,Centre de ressources pour la remédiation cogni...,2,https://aurehal.archives-ouvertes.fr/structure...
3,1077258,Faculté de Santé,1,https://aurehal.archives-ouvertes.fr/structure...
2,1049198,Service de Psychopathologie [Bobigny],1,https://aurehal.archives-ouvertes.fr/structure...
20,1004388,Department of Biochemistry and Genetics [Paris],1,https://aurehal.archives-ouvertes.fr/structure...


## Trier le tableau par ordre décroissant de nombre de documents

In [10]:
df.sort_values(by = ["nb_docs"], ascending =  False)[:4]

,structId,label,nb_docs
13,1062246,"FHU NeuroVasc [Site Sainte-Anne, Paris] [GHU-PPN]",16
14,1008089,geoteca [geoteca],9
9,1072632,Ecole doctorale Médicament -Toxicologie - Chim...,6
1,1007985,European Reference Network on respiratory dise...,3


In [12]:
df_sorted = df.sort_values(by = ["nb_docs"], ascending =  False)

In [13]:
#exporter au format .csv

In [14]:
df_sorted.to_csv("incoming_struct.csv", index = False)

----
## Allez plus loin

ajouter une colonne "last update structure" (champs `updateDate_s` de l'API)